# Iris dataset classification with Neural Network

#### Load the necessary libraries

In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

#### Load the dataset

In [2]:
data = pd.read_csv('Iris.csv')

data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Separate the features and target

In [3]:
# shuffle the dataset
data = data.sample(frac = 1)

x = data.drop(['Id', 'Species'], axis = 1)
y = data.Species

#### Encode the target

In [4]:
encoder = OneHotEncoder(sparse = False)

y = encoder.fit_transform(np.array(y).reshape(-1, 1))

# print first five to view
y[:5]

C:\Users\dasam\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

### Split data into training and testing

In [5]:
# Testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# Validation dataset
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.15)

x_test.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
143,6.8,3.2,5.9,2.3
27,5.2,3.5,1.5,0.2
130,7.4,2.8,6.1,1.9
141,6.9,3.1,5.1,2.3
4,5.0,3.6,1.4,0.2


### Initialize weights

In [6]:
def init_weights (nodes):
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        w = [
                [np.random.uniform(-1, 1) for k in range(nodes[i - 1] + 1)]
                for j in range(nodes[i])
            ]
        
        weights.append(np.matrix(w))
    
    return weights

### Define Activation functions

In [7]:
def sigmoid(x):
    output = 1 / (1 + np.exp(-x))
    return output

def sigmoid_derivative(x):
    output = np.multiply(x, 1 - x)
    return output

### Define function for forward propagation

In [8]:
def Forward_Propagation (x, weights, layers):
    activations, layer_input = [x], x
    
    for i in range (layers):
        activation_output = sigmoid(np.dot(layer_input, weights[i].T))
        activations.append(activation_output)
        
        layer_input = np.append(1, activation_output)
        
    return activations

### Define function for backward propagation

In [9]:
def Backward_Propagation (y, activations, weights, layers):
    output_final = activations[-1]
    
    error = np.matrix(y - output_final)
    
    for i in range(layers, 0, -1):
        current_activation = activations[i]
        
        if (i > 1):
            
            # augment previous activation
            prev_activation = np.append(1, activations[i - 1])
    
        else:
            
            # first hidden layer, prev_activation is input (without bias)
            prev_activation = activations[0]
            
        delta = np.multiply(error, sigmoid_derivative(current_activation))
        weights[i - 1] += alpha * np.multiply(delta.T, prev_activation)
        
        # remove bias from weights
        w = np.delete(weights[i - 1], [0], axis = 1)
        
        # calculate error from current layer
        error = np.dot(delta, w)
        
    return weights

### Train the model

In [10]:
def train(x_input, y_input, alpha, weights):
    layers = len(weights)
    
    for i in range(len(x_input)):
        x, y = x_input.iloc[i], y_input[i]
        x = np.matrix(np.append(1, x))
        
        activations = Forward_Propagation(x, weights, layers)
        weights = Backward_Propagation(y, activations, weights, layers)
        
    return weights

### Define function for prediction

In [11]:
def findMaxActivation(output):
    m, index = output[0], 0
    
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
            
    return index

def predict(item, weights):
    layers = len(weights)
    item = np.append(1, item)
    
    # forward propagation
    activations = Forward_Propagation(item, weights, layers)
    
    output_final = activations[-1].A1
    index = findMaxActivation(output_final)
    
    # intialize prediction vector to zeros
    y = [0 for i in range(len(output_final))]
    y[index] = 1
    
    return y

#### Define function to find accuracy

In [12]:
def accuracy(x_input, y_input, weights):
    correct = 0
    
    for i in range(len(x_input)):
        x, y = x_input.iloc[i], list(y_input[i])
        prediction = predict(x, weights)
        
        if (y == prediction):
            correct += 1
            
    return np.round(correct / len(x_input), 2)

### Compile the Neural Network

In [13]:
def Neural_Network(x_train, y_train, x_val = None, y_val = None, epochs = 10, nodes = [], alpha = 0.2):
    hidden_layers = len(nodes) - 1
    weights = init_weights(nodes)
    
    for epoch in range(1, epochs + 1):
        weights = train(x_train, y_train, alpha, weights)
        
        if (epoch % 20 == 0 or epoch == 1):
            print('Epoch {}'.format(epoch))
            print('Training Accuracy = {}'.format(accuracy(x_train, y_train, weights)))
            
            # validation accuracy
            x1_val = np.array(x_val)
            
            if x1_val.any():
                print('Validation Accuracy = {}'.format(accuracy(x_val, y_val, weights)))
                
            print()
            print('========================================================')
            print()
            
    return weights

#### Perform the training

In [14]:
num_features = len(x.iloc[0])
num_classes = len(y[0])

# number of nodes in layers
layers = [num_features, 5, 10, num_classes]

alpha, epochs = 0.2, 100

weights = Neural_Network(x_train, y_train, x_val, y_val, epochs = epochs, nodes = layers, alpha = alpha)

Epoch 1
Training Accuracy = 0.37
Validation Accuracy = 0.28


Epoch 20
Training Accuracy = 0.79
Validation Accuracy = 0.61


Epoch 40
Training Accuracy = 0.92
Validation Accuracy = 0.78


Epoch 60
Training Accuracy = 0.98
Validation Accuracy = 0.94


Epoch 80
Training Accuracy = 0.98
Validation Accuracy = 1.0


Epoch 100
Training Accuracy = 0.95
Validation Accuracy = 0.89




### Accuracy of Test dataset

In [18]:
accuracy(x_test, y_test, weights)

1.0

## Sklearn Implementation

#### Load the libraries

In [21]:
from sklearn.neural_network import MLPClassifier

#### Create instance of the classifier

In [26]:
classifier = MLPClassifier(random_state = 1, max_iter = 1000)

#### Train the model

In [27]:
classifier.fit(x_train, y_train)

MLPClassifier(max_iter=1000, random_state=1)

#### Make predictions

In [28]:
predictions = classifier.predict(x_test)

#### Find the accuracy

In [30]:
classifier.score(x_test, y_test)

1.0